In [4]:
import cv2
import os
import numpy as np
import pycocotools

In [5]:
#xml to dict
#https://m.blog.naver.com/PostView.naver?isHttpsRedirect=true&blogId=pk3152&logNo=221367256441
import xmltodict
import json

In [66]:
# 압축을 풀고 생기는 최상위 디렉토리
DATA_ROOT = '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new'
# 폴더인 경우만 리스트에 추가
folder_list = [i for i in os.listdir(DATA_ROOT) if os.path.isdir(os.path.join(DATA_ROOT, i))]
print(folder_list)

['Bbox_1671', 'Bbox_1685', 'Bbox_1682', 'Bbox_1676', 'Bbox_1678', 'Bbox_1679', 'Bbox_1677', 'Bbox_1683', 'Bbox_1684', 'Bbox_1702', 'Bbox_1705', 'Bbox_1733', 'Bbox_1734', 'Bbox_1760', 'Bbox_1794', 'Bbox_1758', 'Bbox_1793', 'Bbox_1767', 'Bbox_1751', 'Bbox_1769', 'Bbox_1756', 'Bbox_1735', 'Bbox_1732', 'Bbox_1704', 'Bbox_1703', 'Bbox_1768', 'Bbox_1757', 'Bbox_1750', 'Bbox_1759', 'Bbox_1766', 'Bbox_1792', 'Bbox_1795', 'Bbox_1761', 'Bbox_1743', 'Bbox_1788', 'Bbox_1744', 'Bbox_1772', 'Bbox_1786', 'Bbox_1781', 'Bbox_1775', 'Bbox_1721', 'Bbox_1719', 'Bbox_1726', 'Bbox_1710', 'Bbox_1728', 'Bbox_1717', 'Bbox_1774', 'Bbox_1780', 'Bbox_1787', 'Bbox_1773', 'Bbox_1745', 'Bbox_1742', 'Bbox_1789', 'Bbox_1729', 'Bbox_1716', 'Bbox_1711', 'Bbox_1718', 'Bbox_1727', 'Bbox_1720', 'Bbox_1699', 'Bbox_1807', 'Bbox_1800', 'Bbox_1809', 'Bbox_1697', 'Bbox_1690', 'Bbox_1691', 'Bbox_1696', 'Bbox_1808', 'Bbox_1801', 'Bbox_1806', 'Bbox_1698', 'Bbox_1681', 'Bbox_1675', 'Bbox_1672', 'Bbox_1686', 'Bbox_1688', 'Bbox_1689'

In [95]:
# 폴더에서 xml 확장자를 가진 경우만 리스트업
xml_lists = []
for folder in folder_list:
    xml_file = [os.path.join(folder,i) for i in os.listdir(os.path.join(DATA_ROOT, folder)) if i.endswith('.xml')]
    xml_lists.append(*xml_file)

In [76]:
# xml 파일을 읽어서 dict 로 접근할 수 있게 함
with open(os.path.join(DATA_ROOT, xml_file[0]), 'r') as f:
    clean_xml = xmltodict.parse(f.read())
    # 이렇게 하면 일반적으로 사용하던 json dict 로 편하게 사용할 수 있다고 함
    clean_xml = json.dumps(clean_xml)
    clean_xml = json.loads(clean_xml)

# xml 에 들어있던 기본정보들
print(f'clean_xml.keys() : {clean_xml.keys()}')
print(f"clean_xml['annotations'].keys() : {clean_xml['annotations'].keys()}")
print(f"clean_xml['annotations']['version'] : {clean_xml['annotations']['version']}")
print(f"clean_xml['annotations']['meta'].keys() : {clean_xml['annotations']['meta'].keys()}")
print(f"len(clean_xml['annotations']['image']) : {len(clean_xml['annotations']['image'])}")
# 해당 폴더안에 있는 이미지 갯수랑 실제 xml 에 등록되어 있는 갯수랑 맞는지 확인하기 위함
print(f"image counts : {len([i for i in os.listdir(os.path.join(DATA_ROOT, xml_file[0].split('/')[0])) if i.endswith('.png')])}")

clean_xml.keys() : dict_keys(['annotations'])
clean_xml['annotations'].keys() : dict_keys(['version', 'meta', 'image'])
clean_xml['annotations']['version'] : 1.1
clean_xml['annotations']['meta'].keys() : dict_keys(['task', 'dumped'])
len(clean_xml['annotations']['image']) : 50
image counts : 50


In [74]:
# 이미지를 위한 정보를 찾아보자
# 가장 첫번째 이미지인 0번째 이미지부터 시작
print(clean_xml['annotations']['image'][0].keys())
target_list = ('car', 'person', 'traffic_light', 'truck', 'bicycle', 'motorcycle', 'bus', 'carrier', 'stroller', 'scooter', 'wheelchair')

for k,v in clean_xml['annotations']['image'][0].items():
    print(f'key : {k}, data_type : {type(v)}')
#box 가 list type 이니까 몇 개인지 파악해보자
print()
print(f"len(clean_xml['annotations']['image'][0]['box']) : {len(clean_xml['annotations']['image'][0]['box'])}")
print(f"box keys : {clean_xml['annotations']['image'][0]['box'][0].keys()}")
print('-'*30)
# list 는 잘못 삭제하면 for loop 이 의도하지 않은대로 동작할 수 있어서 조심해야한다.
temp_list = []
for item in clean_xml['annotations']['image'][0]['box']:
    if item['@label'] in target_list:
        temp_list.append(item) 
    else:
        print(f"del {item['@label']} : {item}")
print('-'*30)
clean_xml['annotations']['image'][0]['box'] = temp_list
print(f"after len(clean_xml['annotations']['image'][0]['box']) : {len(clean_xml['annotations']['image'][0]['box'])}")

dict_keys(['@id', '@height', '@width', '@name', 'box'])
key : @id, data_type : <class 'str'>
key : @height, data_type : <class 'str'>
key : @width, data_type : <class 'str'>
key : @name, data_type : <class 'str'>
key : box, data_type : <class 'list'>

len(clean_xml['annotations']['image'][0]['box']) : 9
box keys : dict_keys(['@occluded', '@ybr', '@xbr', '@ytl', '@xtl', '@label'])
------------------------------
del traffic_sign : {'@occluded': '0', '@ybr': '413.19', '@xbr': '633.93', '@ytl': '374', '@xtl': '600.15', '@label': 'traffic_sign'}
del pole : {'@occluded': '1', '@ybr': '569.1', '@xbr': '518.5', '@ytl': '0', '@xtl': '477.7', '@label': 'pole'}
del tree_trunk : {'@occluded': '0', '@ybr': '574.2', '@xbr': '493.2', '@ytl': '259.5', '@xtl': '441.9', '@label': 'tree_trunk'}
del traffic_sign : {'@occluded': '0', '@ybr': '374.03', '@xbr': '633.5', '@ytl': '331.39', '@xtl': '599.73', '@label': 'traffic_sign'}
------------------------------
after len(clean_xml['annotations']['image'][0][

In [112]:
for k, i in clean_xml['annotations']['image'].items():
    print(k)
    print(i['@name'])
    # print(i)
    for j in i['box']:
        print(j)
        # for k in j:
        #     print(k)
        #     break
        break
    break

AttributeError: 'list' object has no attribute 'items'

In [125]:
target_list = ('car', 'person', 'traffic_light', 'truck', 'bicycle', 'motorcycle', 'bus', 'carrier', 'stroller', 'scooter', 'wheelchair')

def data_clensing(xml_locate, xml_file_name):
    with open(os.path.join(DATA_ROOT, xml_locate, xml_file_name),'r') as f:
        #xml 모양의 str 이어야 동작된다.
        raw_xml = xmltodict.parse(f.read())
        #자주 사용한 json dict 로 변환하기 위함(ordereddict 가 반환되어 그렇슴)
        raw_xml = json.dumps(raw_xml)
        clensed_xml = json.loads(raw_xml)

    before, after = 0,0
    delete_image_list = []
    save_xml = dict(annotations=dict(image=[]))
    for image in clensed_xml['annotations']['image']:
        # for box temp list
        temp_list = []
        before += len(image['box'])
        # image['box'] type == dict : box 가 1개인 경우
        if type(image['box']) != list:
            if image['box']['@label'] in target_list:
                # 1개 남았던 박스가 사라지니까 필요 없는 이미지라는 뜻이므로 이미지를 제거하는게 좋다.
                temp_list.append(image['box'])
        # image['box'] type == list : box 가 여러개인 경우
        else:
            for box in image['box']:
                # if box labels in target_list
                if box['@label'] in target_list:
                    temp_list.append(box)

        check_len = len(temp_list)
        # 만약 temp_list 의 길이가 0이라면 추가해야할 박스가 없으므로 그 이미지는 필요 없는 이미지이다.
        if check_len == 0:
            # 난중에 그 이미지는 제거하려고 리스트에 모아둔다.
            delete_image_list.append(os.path.join(DATA_ROOT, xml_locate, xml_file_name, image['@name']))
            continue

        after += check_len
        image['box'] = temp_list
        save_xml['annotations']['image'].append(image)
    
    print(f'before boxes : {before}\t after boxes : {after}')

    print(f'useless image list : {delete_image_list}')
    # json 만들기
    with open(os.path.join(DATA_ROOT, xml_locate, xml_file_name.split('.')[0]+'.json') ,'w') as ff:
        json.dump(save_xml, ff, indent=4)
    
    print(f"before len(clensed_xml) : {len(clean_xml['annotations']['image'])}")
    print(f"after len(clensed_xml) : {len(save_xml['annotations']['image'])}")

def coco_form():
    coco_format = dict(
        info=dict(
            year=2021, version="1.0", description="Pedestrian", contributor="AI Hub", url="https://aihub.or.kr/aidata/136", date_created='2021-11-26 02:39:00'
        ),
        licenses=[
            dict(id=0, name='', url='')
        ],
        categories=[
            dict(id=0, name='car', supercategory=None),
            dict(id=1, name='person', supercategory=None),
            dict(id=2, name='traffic_light', supercategory=None),
            dict(id=3, name='truck', supercategory=None),
            dict(id=4, name='bicycle', supercategory=None),
            dict(id=5, name='motorcycle', supercategory=None),
            dict(id=6, name='bus', supercategory=None),
            dict(id=7, name='carrier', supercategory=None),
            dict(id=8, name='stroller', supercategory=None),
            dict(id=9, name='scooter', supercategory=None),
            dict(id=10, name='wheelchair', supercategory=None)
        ],
        images=[
            dict(license=0, url=None, file_name='', height=512, width=512, data_captured=None, id=0)
        ],
        annotations=[
            dict(id=0, image_id=0, category_id=0, bbox=[], area=1, segmentation=[], iscrowd=0)
        ]
    )

    with open('coco_format.json','w') as f:
        json.dump(coco_format, f, indent=4)


In [126]:
coco_form()

In [91]:
data_clensing(*'Bbox_1802/P1009_28.xml'.split('/'))

before boxes : 529	 after boxes : 297
useless image list : []


In [118]:
data_clensing(*'Bbox_1712/P1005_28.xml'.split('/'))

before boxes : 446	 after boxes : 208
useless image list : ['/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1712/P1005_28.xml/MP_SEL_273375_P025312.jpg']
before len(clensed_xml) : 50
after len(clensed_xml) : 49


In [103]:
for xml in xml_lists:
    print(xml)
    # 얘네가 사라질만 했다는 검증을 어떻게 하징.........?
    data_clensing(*xml.split('/'))

1730/P1007_16.xml/MP_SEL_274432_P026232.jpg', '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1730/P1007_16.xml/MP_SEL_274433_P026233.jpg']
Bbox_1798/P1009_24.xml
before boxes : 414	 after boxes : 192
useless image list : ['/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1798/P1009_24.xml/ZED2_KSC_023924_L_P004159.png', '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1798/P1009_24.xml/ZED2_KSC_023940_L_P004160.png']
Bbox_1753/P1008_09.xml
before boxes : 176	 after boxes : 61
useless image list : ['/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1753/P1008_09.xml/ZED4_KSC_039802_L_P001903.png', '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1753/P1008_09.xml/ZED4_KSC_039807_L_P001904.png', '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1753/P1008_09.xml/ZED4_KSC_039885_L_P001913.png', '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1753/P1008_09.xml/ZED4_KSC_039887_L_P001914.png', '/Users/sm/Downloads/인도보행 영상/바운딩박스/Bbox_21_new/Bbox_1753/P1008_09.xml/ZED4_KSC_

## 앞으로 해야 할 것
- 원하는 정보만 추출해낸 이미지의 아이디를 어떻게 부여 할 것인가
- COCO Format 에 필요한 정보들을 가져와서 json type 으로 만들 것